### Example raw sensor data loading (ECG, ACC)

In [1]:
from sensors.data_loading.data_loader import load_sensor_data
import os

#load ECG data from a start_time to an end_time for a specific participant
data_path = "../data_examples/raw_data"
participant_id = "SMART_999"
subject_dir = os.path.join(data_path, participant_id)
start_time = "20221227_0300"

ecg_data = load_sensor_data(subject_dir, "vivalnk_vv330_ecg", start_time)
acc_data = load_sensor_data(subject_dir, "vivalnk_vv330_acc", start_time)
resp_data = load_sensor_data(subject_dir, "vivalnk_vv330_resp_rate", start_time)


#### Some plotting of the loaded signal

In [2]:
import plotly.graph_objects as go

plot_for = 20 #seconds
fs_ecg = 128
fs_acc = 25

# Convert epoch time to relative time starting from 0 (in seconds)
def convert_to_relative_time(epoch_times):
    start_time = epoch_times[0]
    return [(t - start_time)/1000 for t in epoch_times] 


# Function to create a figure with traces
def create_figure(traces, title, xaxis_title, yaxis_title):
    fig = go.Figure()
    for trace in traces:
        fig.add_trace(go.Scatter(
            x=trace['x'],
            y=trace['y'],
            mode='lines',
            name=trace['name']
        ))
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        template="plotly_dark"
    )
    return fig

# Data for fig1
ecg_traces = [{
    'x': convert_to_relative_time(ecg_data['value.time'][0:fs_ecg*plot_for]),
    'y': ecg_data['value.ecg'][0:fs_ecg*plot_for],
    'name': 'ECG Data'
}]

# Data for fig2
acc_traces = [
    {'x': convert_to_relative_time(acc_data['value.time'][0:fs_acc*plot_for]), 'y': acc_data['value.x'][0:fs_acc*plot_for], 'name': 'x'},
    {'x': convert_to_relative_time(acc_data['value.time'][0:fs_acc*plot_for]), 'y': acc_data['value.y'][0:fs_acc*plot_for], 'name': 'y'},
    {'x': convert_to_relative_time(acc_data['value.time'][0:fs_acc*plot_for]), 'y': acc_data['value.z'][0:fs_acc*plot_for], 'name': 'z'}
]

# Data for fig3
resp_trace = [
    {'x': convert_to_relative_time(resp_data['value.time'][0:5*plot_for]), 'y': resp_data['value.rr'][0:plot_for], 'name': 'x'},
]

# Create figures
fig1 = create_figure(ecg_traces, "Raw ECG", "Time", "ECG Value")
fig2 = create_figure(acc_traces, "Raw ACC", "Time", "ACC Value")
fig3 = create_figure(resp_trace, "Derived Respiration Rate (sensor data)", "Time", "RR")

# Show the plots
fig1.show()
fig2.show()
fig3.show()

